In [4]:
import requests
from psycopg2 import connect, sql

# Configure your PostgreSQL connection string
conn_string = "dbname='etl_bites' user='jackdench' host='localhost' port='5432'"

def get_data_from_api(url):
    response = requests.get(url)
    return response.json()

todos_url = "https://jsonplaceholder.typicode.com/todos"
users_url = "https://jsonplaceholder.typicode.com/users"

todos_data = get_data_from_api(todos_url)
users_data = get_data_from_api(users_url)


In [5]:
def join_todos_and_users(todos, users):
    completed_tasks = {}
    for todo in todos:
        user_id = todo['userId']
        if user_id not in completed_tasks:
            completed_tasks[user_id] = 0
        if todo['completed']:
            completed_tasks[user_id] +=1
    result = []
    for user in users:
        result.append({'userId': user['id'], 'name': user['name'], 'completedTasks': completed_tasks[user['id']]})
    
    return result

combined_data = join_todos_and_users(todos_data, users_data)
print(combined_data)

[{'userId': 1, 'name': 'Leanne Graham', 'completedTasks': 11}, {'userId': 2, 'name': 'Ervin Howell', 'completedTasks': 8}, {'userId': 3, 'name': 'Clementine Bauch', 'completedTasks': 7}, {'userId': 4, 'name': 'Patricia Lebsack', 'completedTasks': 6}, {'userId': 5, 'name': 'Chelsey Dietrich', 'completedTasks': 12}, {'userId': 6, 'name': 'Mrs. Dennis Schulist', 'completedTasks': 6}, {'userId': 7, 'name': 'Kurtis Weissnat', 'completedTasks': 9}, {'userId': 8, 'name': 'Nicholas Runolfsdottir V', 'completedTasks': 11}, {'userId': 9, 'name': 'Glenna Reichert', 'completedTasks': 8}, {'userId': 10, 'name': 'Clementina DuBuque', 'completedTasks': 12}]


In [6]:
# Create tables in analytical DB
# This could also be done manually via a GUI (e.g. TablePlus) or with a SQL script
def execute_query_postgresql(conn_string, query):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

create_api_data_table = '''
CREATE TABLE api_data_2 (
    userId INTEGER,
    name TEXT,
    completedTasks INTEGER
);
'''

execute_query_postgresql(conn_string, create_api_data_table)

In [7]:
def insert_data_to_postgresql(conn_string, table_name, data):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            for item in data:
                query = sql.SQL("INSERT INTO {} (userId, name, completedTasks) VALUES (%s, %s, %s)").format(sql.Identifier(table_name))
                cur.execute(query, (item['userId'], item['name'], item['completedTasks']))
        conn.commit()

table_name = "api_data_2"
insert_data_to_postgresql(conn_string, table_name, combined_data)